In [ ]:
import glob
import os
import itertools
import time
import warnings

import pandas as pd
import pandarallel
import numpy as np
import tqdm

import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.stats.multitest

warnings.filterwarnings("ignore")
np.seterr(all="ignore")

pandarallel.pandarallel.initialize(nb_workers=14)

INFO: Pandarallel will run on 14 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


### 1. Data Preprocessing
Perform normal/abnormal binarization of factors in the sample based on clinical standard information.

In [ ]:
def clinical_binary(sample: pd.Series):
    sample['B_PRT16_U'] = 0 if sample['B_PRT16_U'] == 0 else 1
    sample['B_GLU16_U'] = 0 if sample['B_GLU16_U'] == 0 else 1
    sample['B_BLOOD16_U'] = 0 if sample['B_BLOOD16_U'] == 0 else 1
    sample['B_HBA1C'] = 0 if sample['B_HBA1C'] < 5.6 else 1
    sample['B_GLU0'] = 0 if sample['B_GLU0'] > 70 and sample['B_GLU0'] < 99 else 1
    sample['B_BUN'] = 0 if sample['B_BUN'] > 70 and sample['B_BUN'] < 99 else 1
    sample['B_ALBUMIN'] = 0 if sample['B_ALBUMIN'] > 3.58 and sample['B_ALBUMIN'] < 5.2 else 1
    sample['B_T_BIL'] = 0 if sample['B_T_BIL'] < 1.2 else 1
    sample['B_TCHL'] = 0 if sample['B_TCHL'] < 199 else 1
    sample['B_HDL'] = 0 if sample['B_HDL'] > 40 and sample['B_HDL'] < 60 else 1
    sample['B_LDL'] = 0 if sample['B_LDL'] < 100 else 1
    sample['B_TG'] = 0 if sample['B_TG'] < 149 else 1
    sample['B_WBC_B'] = 0 if sample['B_WBC_B'] > 4 and sample['B_WBC_B'] < 10 else 1
    sample['B_PLAT'] = 0 if sample['B_PLAT'] > 150 and sample['B_PLAT'] < 370 else 1

    if sample['B_SEX'] == 1:
        sample['B_CREATINE'] = 0 if sample['B_CREATINE'] > 0.7 and sample['B_CREATINE'] < 1.2 else 1
        sample['B_AST'] = 0 if sample['B_AST'] < 40 else 1
        sample['B_ALT'] = 0 if sample['B_ALT'] < 41 else 1
        sample['B_R_GTP'] = 0 if sample['B_R_GTP'] > 10 and sample['B_R_GTP'] < 71 else 1
        sample['B_RBC_B'] = 0 if sample['B_RBC_B'] > 4.1 and sample['B_RBC_B'] > 5.6 else 1
        sample['B_HB'] = 0 if sample['B_HB'] > 13 and sample['B_HB'] < 17 else 1
        sample['B_HCT'] = 0 if sample['B_HCT'] > 39 and sample['B_HCT'] < 51 else 1

    elif sample['B_SEX'] == 2:
        sample['B_CREATINE'] = 0 if sample['B_CREATINE'] > 0.5 and sample['B_CREATINE'] < 0.9 else 1
        sample['B_AST'] = 0 if sample['B_AST'] < 32 else 1
        sample['B_ALT'] = 0 if sample['B_ALT'] < 33 else 1
        sample['B_R_GTP'] = 0 if sample['B_R_GTP'] > 6 and sample['B_R_GTP'] < 42 else 1
        sample['B_RBC_B'] = 0 if sample['B_RBC_B'] > 3.7 and sample['B_RBC_B'] > 4.7 else 1
        sample['B_HB'] = 0 if sample['B_HB'] > 11 and sample['B_HB'] < 15 else 1
        sample['B_HCT'] = 0 if sample['B_HCT'] > 33 and sample['B_HCT'] < 45 else 1

    return sample

1-1. Lists disease groups and clinical information to be verified.

In [63]:
basics = ['B_SEX']
diseases = ['B_HTN', 'B_DM', 'B_THY', 'B_LIP', 'B_LCA',
            'B_GCA', 'B_HCCCA', 'B_COLCA', 'B_PACA', 'B_UTCA',
            'B_BRCA', 'B_THYCA', 'B_PROCA', 'B_GALLCA']
clinicals = ['B_BMI', 'B_PRT16_U', 'B_GLU16_U', 'B_BLOOD16_U', 'B_HBA1C',
             'B_GLU0', 'B_BUN', 'B_ALBUMIN', 'B_CREATINE', 'B_AST',
             'B_T_BIL', 'B_ALT', 'B_TCHL', 'B_R_GTP', 'B_HDL',
             'B_LDL', 'B_TG', 'B_WBC_B', 'B_RBC_B', 'B_HB',
             'B_HCT', 'B_PLAT']

1-2. The selected clinical information is binarized based on clinical standards.

In [66]:
raw_data = pd.read_csv('data/KOGES_BASE.csv')
raw_data = raw_data.set_index('RID')
raw_data = raw_data.replace([66666, 77777, 99999], np.nan)
raw_data = raw_data.loc[:, basics + diseases + clinicals].copy()
raw_data = raw_data.dropna()

raw_data = raw_data.parallel_apply(clinical_binary, axis=1, result_type='expand')

raw_data.loc[:, diseases] = raw_data.loc[:, diseases].replace([1, 2], [0, 1])

raw_data.loc[:, 'Obesity'] = (raw_data.loc[:, 'B_BMI'] > 30).astype('int')
raw_data = raw_data.drop('B_BMI', axis=1)
raw_data = raw_data.drop('B_SEX', axis=1)

clinicals.remove('B_BMI')
diseases.append('Obesity')

raw_data = raw_data.astype('bool') 

### 2. Network constructing

Based on the binarized clinical information, we analyze the correlation of each information according to the Novelty-clustering algorithm.


In [7]:
def overlap_ratio(x: pd.Series, GAM: pd.DataFrame):
    a = GAM.loc[x['gene_1'], :]
    b = GAM.loc[x['gene_2'], :]
    if (x['n_1'] == 0) | (x['n_2'] == 0):
        return 0
    return (a * b).abs().sum() / min(x['n_1'], x['n_2'])

def H_1(x: pd.Series, GAM: pd.DataFrame):
    import numpy as np
    a = GAM.loc[x['gene_1'], :]
    dist = np.array([(a == 0).sum(), (a == 1).sum()])
    dist = dist / GAM.shape[1]
    dist[dist == 0] = 1
    return (-dist * np.emath.logn(2, dist)).sum()

def H_2(x: pd.Series, GAM: pd.DataFrame):
    import numpy as np
    a = GAM.loc[x['gene_2'], :]
    dist = np.array([(a == 0).sum(), (a == 1).sum()])
    dist = dist / GAM.shape[1]
    dist[dist == 0] = 1
    return (-dist * np.emath.logn(2, dist)).sum()

def H_1_2(x: pd.Series, GAM: pd.DataFrame):
    import numpy as np
    a = GAM.loc[x['gene_1'], :]
    b = GAM.loc[x['gene_2'], :]
    dist = np.array([[((a == 0) & (b == 0)).sum(), ((a == 1) & (b == 0)).sum()],
                     [((a == 0) & (b == 1)).sum(), ((a == 1) & (b == 1)).sum()]])
    dist = dist.flatten() / GAM.shape[1]
    dist[dist == 0] = 1
    return (-dist * np.emath.logn(2, dist)).sum()

def w(x: pd.Series, params=(15, 0.5)):
    import numpy as np
    f = max(x['freq_1'], x['freq_2'])
    return 1 / (1 + np.exp(-params[0] * (f - params[1])))

def weighted_entropy(x: pd.Series, GAM: pd.DataFrame):
    import numpy as np
    w = x['w']
    a = GAM.loc[x['gene_1'], :]
    b = GAM.loc[x['gene_2'], :]

    dist_1 = np.array([sum(a == 0), sum(a == 1) / w])
    dist_1 = dist_1 / dist_1.sum()
    dist_1[dist_1 == 0] = 1
    H_1 = (-dist_1 * np.emath.logn(2, dist_1)).sum()

    dist_2 = np.array([sum(b == 0), sum(b == 1) / w])
    dist_2 = dist_2 / dist_2.sum()
    dist_2[dist_2 == 0] = 1
    H_2 = (-dist_2 * np.emath.logn(2, dist_2)).sum()

    dist_1_2 = np.array([[((a == 0) & (b == 0)).sum(), ((a == 1) & (b == 0)).sum() / w],
                         [((a == 0) & (b == 1)).sum() / w, ((a == 1) & (b == 1)).sum() / w]])
    dist_1_2 = dist_1_2.flatten() / dist_1_2.sum()
    dist_1_2[dist_1_2 == 0] = 1
    H_1_2 = (-dist_1_2 * np.emath.logn(2, dist_1_2)).sum()
    
    return (H_1_2, H_1 + H_2 - H_1_2)

def ASC(x: pd.Series, pseudo_result: pd.DataFrame):
    pseudo_result = pseudo_result.copy()
    term1 = pseudo_result.loc[(pseudo_result.loc[:, 'gene_1'] == x['gene_1']) | (pseudo_result.loc[:, 'gene_2'] == x['gene_1']),
                             'effect_size'].mean()
    term2 = pseudo_result.loc[(pseudo_result.loc[:, 'gene_1'] == x['gene_2']) | (pseudo_result.loc[:, 'gene_2'] == x['gene_2']),
                             'effect_size'].mean()
    term3 = pseudo_result.loc[:, 'effect_size'].mean()
    return term1 + term2 - term3

Parameter setup

In [8]:
randomized_count = 1000
weight_params = (15, 0.15)
fdr_alpha = 0.1

Run the novelty-algorithm

In [ ]:
metabolic_syndromes = ['B_LIP', 'Obesity']

for disease in metabolic_syndromes:
    # single disease
    # alteration_matrix = raw_data.loc[raw_data.loc[:, disease], clinicals].copy()

    # metabolic syndrome with complication
    alteration_matrix = raw_data.loc[(raw_data.loc[:, ['B_HTN', 'B_DM', 'B_HTN', 'B_LIP', 'B_THY', 'Obesity']].sum(axis=1) >= 2) & raw_data.loc[:, disease], clinicals].copy()
    alteration_matrix.index.name = ''
    alteration_matrix = alteration_matrix.transpose()

    result = pd.DataFrame(itertools.combinations(alteration_matrix.index, 2), columns=['gene_1', 'gene_2'])
    result.loc[:, 'n_1'] = alteration_matrix.loc[result.loc[:, 'gene_1'], :].isin([1]).sum(axis=1).values
    result.loc[:, 'n_2'] = alteration_matrix.loc[result.loc[:, 'gene_2'], :].isin([1]).sum(axis=1).values
    result.loc[:, 'freq_1'] = result.loc[:, 'n_1'] / alteration_matrix.shape[1]
    result.loc[:, 'freq_2'] = result.loc[:, 'n_2'] / alteration_matrix.shape[1]
    result.loc[:, 'overlap_ratio'] = result.parallel_apply(overlap_ratio, GAM=alteration_matrix, axis=1)
    
    # drop pairs with 0 overlap ratio to decrease calculation time
    result = result.loc[result.loc[:, 'overlap_ratio'] != 0, :].copy()
    result = result.reset_index(drop=True)
    result.loc[:, 'H_1'] = result.parallel_apply(H_1, GAM=alteration_matrix, axis=1)
    result.loc[:, 'H_2'] = result.parallel_apply(H_2, GAM=alteration_matrix, axis=1)
    result.loc[:, 'H_1_2'] = result.parallel_apply(H_1_2, GAM=alteration_matrix, axis=1)
    result.loc[:, 'MI'] = result.loc[:, 'H_1'] + result.loc[:, 'H_2'] - result.loc[:, 'H_1_2']
    result.loc[:, 'w'] = result.parallel_apply(w, params=weight_params, axis=1) # choose rho as the median value of frequency
    result.loc[:, 'wH_1_2'], result.loc[:, 'wMI'] = result.parallel_apply(weighted_entropy, GAM=alteration_matrix, axis=1, result_type='expand').T.values
    
    print(f"[{time.strftime('%c', time.localtime(time.time()))}]\t Statistical testing")
    tmp = result.loc[:, ['gene_1', 'gene_2', 'w']].copy()
    
    for i in tqdm.tqdm(range(0, randomized_count)):
        np.random.seed(i)
        GAM_random = alteration_matrix.copy()
        GAM_random.apply(lambda x: np.random.shuffle(x.values), axis=1)
        tmp = pd.concat([tmp, pd.Series(tmp.parallel_apply(weighted_entropy, GAM=GAM_random, axis=1, result_type='expand').T.values[1])], axis=1)

    result.loc[:, 'pvalue'] = (tmp.loc[:, 0].transpose() > result.loc[:, 'wMI']).sum() / randomized_count
    result.loc[:, 'FDR'], result.loc[:, 'FDR_pvalue'] =\
        statsmodels.stats.multitest.fdrcorrection(result.loc[:, 'pvalue'], alpha=fdr_alpha)
    result.loc[:, 'effect_size'] =result.loc[:, 'wMI'] - tmp.loc[:, 0].mean(axis=1)

    result.loc[:, 'ASC'] = result.parallel_apply(ASC, pseudo_result=result, axis=1)
    result.loc[:, 'EIS'] = result.loc[:, 'effect_size'] - result.loc[:, 'ASC']
    result.loc[:, 'norm_EIS'] = result.loc[:, 'EIS'] / result.loc[:, 'wH_1_2']

    result.to_csv(f"{disease}.csv")